In [6]:
import numpy as np
import scipy as sp
import scipy.linalg
import cvxpy as cp
import matplotlib.pyplot as plt
import random
import math
import sys

In [2]:

def generate_data(savefile_bool, p_x, p_y, p_z, tot_samples):

    syndrome_col = np.zeros([tot_samples, n-k])
    error_col = np.zeros([tot_samples, 2*n])

    for i_sample in range(tot_samples):
        # generate qbits randomly
        tx_qbits = np.random.rand(2**k)
        tx_qbits = NormalizeState(tx_qbits)

        # encode qbits
        tx_encoded = encode_qbits(tx_qbits)

        # channel
        rx_erry, error_vector = depolarizing_channel(tx_encoded, p_x, p_y, p_z)

        # syndrome
        syndr = get_syndrome(rx_erry)

        # fill columns
        syndrome_col[i_sample, :] = syndr
        error_col[i_sample, :] = error_vector
        
        if(savefile_bool):
            savefile_name = 'data_' + str(n) + ',' + str(k) + '_tot_samples_'+ str(tot_samples) + '_p_' + str(p_x)
            savefile_name_syndr = savefile_name + '_syndr.csv'
            savefile_name_error = savefile_name + '_error.csv'

            np.savetxt(savefile_name_syndr, syndrome_col, delimiter=",")
            np.savetxt(savefile_name_error, error_col, delimiter=",")

            print('Saved files:')
            print(savefile_name_syndr)
            print(savefile_name_error)
        
    return syndrome_col, error_col


In [11]:
Id = np.eye(2)
X = np.array([[0.0, 1.0],[1.0, 0.0]])
Z = np.array([[1.0, 0.0],[0.0, -1.0]])
Y = np.matmul(X,Z)

zero = np.array([[1.0], [0.0]]) # |0>
one = np.array([[0.0], [1.0]]) # |1>

In [12]:
def NormalizeState(ipVal):
    if(sp.linalg.norm(ipVal) == 0): return ipVal
    else : return ipVal / sp.linalg.norm(ipVal)
    
def NKron(*args):
  result = np.array([[1.0]])
  for op in args:
    result = np.kron(result, op)
  return result
    
def NKronModified(checkRowMod):
  result = np.array([[1.0]])
  for ind in checkRowMod:
    if(ind == 0):
        op = Id
    elif(ind == 1):
        op = X
    elif(ind == 2):
        op = Y
    elif(ind == 3):
        op = Z
    result = np.kron(result, op)
  return result

def getGenerator(checkRow):
    checkRowModified = np.zeros(n, dtype=int)
    
    checkRowModified[(checkRow[:n] == checkRow[n:]) & (checkRow[n:] == 1)] = 2
    checkRowModified[(checkRow[:n] == 1) & (checkRowModified != 2)] = 1
    checkRowModified[(checkRow[n:] == 1) & (checkRowModified != 2)] = 3
    
    return NKronModified(checkRowModified)  

In [13]:
# checkMatrix = np.array([[0,0,0,1,1,1,1, 0,0,0,0,0,0,0],
#                         [0,1,1,0,0,1,1, 0,0,0,0,0,0,0],
#                         [1,0,1,0,1,0,1, 0,0,0,0,0,0,0],
#                         [0,0,0,0,0,0,0, 0,0,0,1,1,1,1],
#                         [0,0,0,0,0,0,0, 0,1,1,0,0,1,1],
#                         [0,0,0,0,0,0,0, 1,0,1,0,1,0,1]])

checkMatrix = np.array([[1,0,0,1,0, 0,1,1,0,0],
                        [0,1,0,0,1, 0,0,1,1,0],
                        [1,0,1,0,0, 0,0,0,1,1],
                        [0,1,0,1,0, 1,0,0,0,1]])

n = int(checkMatrix.shape[1]/2)
k = n-checkMatrix.shape[0]

gi = np.zeros([n-k, 2**n, 2**n])
for i in range(n-k):
    gi[i,:,:] = getGenerator(checkMatrix[i,:])

########## G Matrix ##########
Gmatrix = np.eye(gi[0,:,:].shape[0], gi[0,:,:].shape[1]) # generator matrix corresponding to this code
for i in range(n-k):
    Gmatrix = Gmatrix + np.matmul(gi[i,:,:], Gmatrix)
Gmatrix = np.round(Gmatrix)

########## Non-zero unique columns ##########
# get boolean array if the columns are zero or not
zeroCols = np.zeros(Gmatrix.shape[1])
for i in range(Gmatrix.shape[1]):
    zeroCols[i] = all(Gmatrix[:,i] == np.zeros(Gmatrix.shape[0]))

# get indices of non-zero columns
nonZeroColsList = np.argwhere(zeroCols==0).flatten()

# get all non zero columns
GmatrixNonZero = np.zeros([Gmatrix.shape[0], nonZeroColsList.shape[0]])
i = 0
for ind in nonZeroColsList:
    GmatrixNonZero[:,i] = Gmatrix[:,ind]
    i = i+1

# get all non zero and unique columns and there indices
GmatrixNonZeroUniqueInd, nonZeroUniqueInd = np.unique(GmatrixNonZero, axis = 1, return_index=True)
nonZeroUniqueInd = nonZeroColsList[nonZeroUniqueInd]

In [15]:
def encode_qbits(qbits):
    # get extended qbits corresponding to non-zero column indices of G matrix
    encoded = np.zeros(2**n)
    i = 0
    for nonZeroIndex in np.sort(nonZeroUniqueInd):
        if(i>=2**k):
            break
        encoded[nonZeroIndex] = qbits[i]
        i = i+1
    encoded = NormalizeState(encoded)

    # encode transmit qbits using generators
    for i in range(n-k):
        encoded = encoded + np.matmul(gi[i,:,:], encoded)
    encoded = NormalizeState(encoded)
    
    return encoded

def depolarizing_channel(input_qbits, p_x, p_y, p_z):
    p_channel = p_channel = [1-p_x-p_y-p_z, p_x, p_y, p_z]  
    errMatrix = np.random.multinomial(1, p_channel, size=n)
    error_vector = errMatrix@np.array([0,1,2,3])
    channel_error = NKronModified(error_vector)
    
    output_qbits = np.dot(channel_error, input_qbits)
    
    error_vector = np.append(error_vector.reshape([n,1]), np.zeros([n,1]), axis=1)
    
    error_vector[error_vector[:,0] == 1, 1] = 1
    error_vector[error_vector[:,0] == 3, 1] = 1
    error_vector[error_vector[:,0] == 1, 0] = 0
    error_vector[error_vector[:,0] == 2, 0] = 1
    error_vector[error_vector[:,0] == 3, 0] = 1
    
    return output_qbits, error_vector.flatten()

def get_syndrome(input_qbits):
    syndr = np.zeros(n-k)
    for i in range(n-k):
        syndr[i] = np.dot(input_qbits.transpose(), np.dot(gi[i,:,:], input_qbits))
    syndr = syndr.flatten() 
    
    syndr[syndr>0] = 0
    syndr[syndr<0] = 1    
    
    return syndr

def generate_data(savefile_bool, p_x, p_y, p_z, tot_samples):

    syndrome_col = np.zeros([tot_samples, n-k])
    error_col = np.zeros([tot_samples, 2*n])

    for i_sample in range(tot_samples):
        # generate qbits randomly
        tx_qbits = np.random.rand(2**k)
        tx_qbits = NormalizeState(tx_qbits)

        # encode qbits
        tx_encoded = encode_qbits(tx_qbits)

        # channel
        rx_erry, error_vector = depolarizing_channel(tx_encoded, p_x, p_y, p_z)

        # syndrome
        syndr = get_syndrome(rx_erry)

        # fill columns
        syndrome_col[i_sample, :] = syndr
        error_col[i_sample, :] = error_vector
        
        if(savefile_bool):
            savefile_name = 'data_' + str(n) + ',' + str(k) + '_tot_samples_'+ str(tot_samples) + '_p_' + str(p_x)
            savefile_name_syndr = savefile_name + '_syndr.csv'
            savefile_name_error = savefile_name + '_error.csv'

            np.savetxt(savefile_name_syndr, syndrome_col, delimiter=",")
            np.savetxt(savefile_name_error, error_col, delimiter=",")

            print('Saved files:')
            print(savefile_name_syndr)
            print(savefile_name_error)
        
    return syndrome_col, error_col

def build_decoder(tot_layers, hidden_dim, hidden_actvn, output_actvn, loss_func, optimizer, metrics):
    model = Sequential()
    
    model.add(Dense(hidden_dim, activation=hidden_actvn, input_shape=(n-k,), name = 'layer_0'))
    for layers in range(tot_layers-1):
        model.add(Dense(hidden_dim, activation=hidden_actvn, name = 'layer_' + str(layers+1)))
        model.add(BatchNormalization(name='normalize_'+str(layers+1), trainable=True))
    model.add(Dense(2*n, activation=output_actvn, name = 'output_layer'))
    
    model.compile(loss=loss_func, optimizer=optimizer, metrics=metrics)
    
    return model

def train_model(p_x, p_y, p_z, tot_samples, tot_layers, hidden_dim, hidden_actvn, output_actvn, loss_func, optimizer, metrics, verbose_bool, tot_epochs):
    syndromes, errors = generate_data(False, p_x, p_y, p_z, tot_samples)
    X_train = syndromes
    y_train = errors

    # process data for argmax P(e/s)

    X_train_unique = np.unique(X_train, axis=0)
    y_train_unique = np.zeros([X_train_unique.shape[0], y_train.shape[1]])

    temp_ind = 0
    for train_sample in X_train_unique:
        temp_error_list = y_train[np.all(X_train == train_sample, axis=1)]  
        temp_error_list_unique, counts = np.unique(temp_error_list, axis=0, return_counts=True)
        y_train_unique[temp_ind] = temp_error_list_unique[counts == max(counts)][0,:].flatten()
        temp_ind = temp_ind + 1

    model = build_decoder(tot_layers, hidden_dim, hidden_actvn, output_actvn, loss_func, optimizer, metrics)
    model.fit(X_train_unique, y_train_unique, epochs=500, verbose=verbose_bool)
    
    return model

In [23]:
tot_iter = 1000
tot_iter_algo = 10

tot_probabs = 25
max_probab = 1-0.01-0.01
# p_y = a_y*p_x + b_y
a_y = 0
b_y = 0
# p_z = a_z*p_x + b_z
a_z = 0
b_z = 0

# train model parameters
tot_samples = 10000

tot_layers=5
hidden_dim=100
hidden_actvn='relu'
output_actvn='sigmoid'
learning_rate = 0.01
tot_epochs = 500

# optimizer = Adam(learning_rate)
loss_func='binary_crossentropy' # binary_crossentropy # mean_squared_error
metrics=['binary_crossentropy', 'acc']
verbose_bool = 0

In [24]:
p_x = 0.1
p_y = a_y*p_x + b_y
p_z = a_z*p_x + b_z

syndromes, errors = generate_data(False, p_x, p_y, p_z, tot_samples)
X_train = syndromes
y_train = errors

# process data for argmax P(e/s)

X_train_unique = np.unique(X_train, axis=0)
y_train_unique = np.zeros([X_train_unique.shape[0], y_train.shape[1]])

temp_ind = 0
for train_sample in X_train_unique:
    temp_error_list = y_train[np.all(X_train == train_sample, axis=1)]  
    temp_error_list_unique, counts = np.unique(temp_error_list, axis=0, return_counts=True)
    y_train_unique[temp_ind] = temp_error_list_unique[counts == max(counts)][0,:].flatten()
    temp_ind = temp_ind + 1

In [25]:
X_train_unique

array([[0., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 1., 1.],
       [0., 1., 0., 0.],
       [0., 1., 0., 1.],
       [0., 1., 1., 0.],
       [0., 1., 1., 1.],
       [1., 0., 0., 0.],
       [1., 0., 0., 1.],
       [1., 0., 1., 0.],
       [1., 0., 1., 1.],
       [1., 1., 0., 0.],
       [1., 1., 0., 1.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.]])

In [42]:
np.unique(y_train[np.all(X_train ==  [1., 0., 1., 0.] , axis=1)], axis=0, return_counts=True)

(array([[0., 0., 0., 0., 0., 1., 0., 1., 0., 0.],
        [0., 1., 0., 1., 0., 0., 0., 0., 0., 1.]]), array([64, 12]))

In [34]:
y_train_unique[3,:]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])

In [7]:
a = np.array([1,2])

In [10]:
file = open('testfile.txt','w') 
 
file.write(str(a)) 
file.write('\n\n\n') 
file.write(str(a)) 
 
file.close() 